# Working with MANTO data

<div class="alert alert-warning">
    <h4>Rate limit on JSON requests to MANTO server</h4>
    <p>In early testing, I've been getting an error from time to time from MANTO saying that the maximum number of requests has been reached for a given 15-minute period. When this happens, empty placeholder Manto Entities are created by the client. Unfortunately, those are then cached, which means that the client doesn't try again to retrieve the missing data.</p>
    <p>For now, I've introduced the <code>cache_empty=False</code> default for <code>getMantoID()</code> and <code>getMantoChar()</code>, which specifies that an entity retrieved from the cache with an empty <code>data</code> attribute should not be cached, forcing it to be redownloaded next time it's used. I'll talk to MANTO about how to solve this on a more permanent basis.</p>
</div>

### Preliminaries

Import statemtents. Our MANTO tools are in the `dicesapi.manto` module.

In [ ]:
from dicesapi import DicesAPI
from dicesapi.jupyter import NotebookPBar
from dicesapi import manto

Create a connection to DICES

In [ ]:
api=DicesAPI(progress_class=NotebookPBar)

Instantiate a DICES character to get started

In [ ]:
ach = api.getCharacters(name='Achilles')[0]
print(ach)

### Retrieving MANTO entities

Let's create a new MANTO entity by retrieving Achilles' ID from MANTO.

In [ ]:
manto_ent = manto.getMantoChar(ach)
print(manto_ent)

### Using MANTO ties to find relatives

This code is still in development and subject to change. But for now, this is how we search MANTO for Achilles' parents. Under the hood, we're checking two different MANTO ties, `31764`, "son of", and `31765`, "daughter of."

This function currently returns MANTO Entity objects.

In [ ]:
parents = manto_ent.getParents()
print(parents)

Let's look for grandparents. Note that when sources disagree about parentage, MANTO returns an inclusive list.

In [ ]:
for p in parents:
    print(f'Grandparents on {p.name}’s side:')
    for gp in p.getParents():
        print(' - ' + gp.name)

### Boolean relationship checker

#### For MANTO Entities

Sometimes we just want a yes/no answer as to parentage. For MANTO Entities, methods like `isChildOf()` will take a second Manto entity as argument and return `True` or `False`. This can be useful, for example, for filtering a list.

In [ ]:
# a list of character pairs, by MANTO ID
pairs = [
    ('8182179', '8182084'),  # Anchises and Aeneas
    ('8188282', '8190182'),  # Eteocles and Oedipus
    ('8189802', '8182035'),  # Menelaus and Agamemnon
]

for a, b in pairs:
    ent_a = manto.getMantoID(a)
    ent_b = manto.getMantoID(b)
    print(f'Is {ent_a.name} the child of {ent_b.name}? ', ent_a.isChildOf(ent_b))

#### For DICES Characters

For convenience, there are also wrapper functions that perform the same checks on DICES Characters by resolving the characters to MANTO entities first, then calling the boolean methods above. If you pass a CharacterInstance instead of a Character, the function will try to determine the MANTO ID of the underlying Character.

If passed a Character with no MANTO ID, or an anonymous CharacterInstance with no Character, these functions return `None`. You can use the optional `err_val` parameter to set a different fallback value; for example, if you're using the function in a filter and want to ensure it always returns a Boolean, set this to `False`.

For these next examples, let's start with a list of Achilles' interlocutors.

In [ ]:
speeches = api.getSpeeches(spkr_name='Achilles', progress=True)
addressees = speeches.getAddrs()

In this block, we use call `CharIsChildOf` on the speaker and his addressees to test parentage using MANTO's data. Where the function returns `None`, we know data was missing.

In [ ]:
for addr in addressees:
    check = manto.CharIsChild(ach, addr)
    if check is not None:
        print(f'Is {ach.name} the child of {addr.name}?', check)
    else:
        print(f'No MANTO data available for {addr.name}')

But if we want to use the function in a custom filter, we can silence the errors and force it to just return `False` when it doesn't find MANTO data.

In [ ]:
# filter a CharacterInstanceGroup based on MANTO ties:
parent_addrs = addressees.advancedFilter(lambda addr: manto.CharIsChild(ach, addr, debug=False, err_val=False))

print('Parent addressees of Achilles:')
for p in parent_addrs:
    print(p.name)
print()
    
# now use the filtered list of CharacterInstances to filter the original SpeechGroup
parent_speeches = speeches.filterAddrInstances(parent_addrs)

print('Speeches in which Achilles addresses a parent:')
for s in parent_speeches:
    print(f'{s.author.name} {s.work.title} {s.l_range}')